# 2 Data Cleaning

This notebook combines the various loose data tables from notebook 1 into a singular flat table for later use. NFTs with null values and / or 0 sales values were removed. In conclusion, a final flat table of 19 columns and 32949 rows of data was produced. Refer to Section 2.7 for preview of the final table.

In [1]:
import requests
import json
import pandas as pd
import datetime
import numpy as np

pd.options.display.max_colwidth = 200

## 2.1 Loading the data

In [2]:
filepath = "../data/raw/"

#data tables
creator_df = pd.read_csv(filepath + 'creator_df.csv')
nft_sales_df = pd.read_csv(filepath + 'nft_sales_df.csv')
nft_bid_df = pd.read_csv(filepath + 'nft_bid_df.csv')
nft_metadata_df = pd.read_csv(filepath + 'nft_metadata_df.csv')

# connectors
creator_nft_df = pd.read_csv(filepath + 'creator_nft_df.csv')
nft_nft_history_df = pd.read_csv(filepath + 'nft_nft_history_df.csv')

## 2.2 Cleaning of creator_df data

In [3]:
creator_df.head()

,id,netRevenueInETH,netRevenuePendingInETH,netSalesInETH,netSalesPendingInETH
0,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.400000,0.0,2224.000000,0.0
1,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,0.0,1624.651700,0.0
2,0x484ec62385e780f2460feac34864a77ba5a18134,513.400000,0.0,604.000000,0.0
3,0x0624d062ae9dd596de0384d37522cde46cd500d6,425.000000,0.0,500.000000,0.0
4,0x0f441cfad93287109f5ef834bf52f4aaaa8d8ffa,380.005263,0.0,447.065016,0.0


According the Foundation API, the 'netRevenuePendingInETH' and 'netSalesPendingInETH' refers to part of total ETH sales for minted NFTs which transactions have not been finalized.  

These columns are more indicative each creator/collectors choice in finalizing ETH transactions, and thus are not included in the analysis of ETH price of NFTs. 

In [4]:
creator_df.drop(['netRevenuePendingInETH', 'netSalesPendingInETH'], axis = 1, inplace = True);

'netRevenueInETH' refers to the value in ETH earned by each creator after subtracting intermediary fees such as ETH gas fees.  
'netSalesInETH' refers to the value in ETH for sales of NFTs by each creator.  

To check, a new column 'c_rs_ratio' is created to check that net sales is always larger than net revenue. 

In [5]:
creator_df['c_rs_ratio'] = creator_df['netRevenueInETH'] / creator_df['netSalesInETH']
creator_df[creator_df['c_rs_ratio'] > 1].shape

(0, 4)

Finally, the prefix "c" is added to each column name to signify info related to creator. The net sales and revenue columns can be used as a ranking system to rank creator by earnings, which could be a variable that affects NFT price.  

Both the sales and revenue columns are kept in as there might be additional factors such as gas price aside from the 15% service fee charged by Foundation.

In [6]:
new_column_names = ['c_id', 'c_revenue', 'c_sales', 'c_rs_ratio']
old_column_names = creator_df.columns
column_change = {old_column_names[col]:new_column_names[col] for col in range(len(old_column_names))}
creator_df.rename(columns = column_change, inplace = True);

In [7]:
creator_df.isnull().sum()

c_id          0
c_revenue     0
c_sales       0
c_rs_ratio    0
dtype: int64

## 2.3 Cleaning of nft_sales_df data

In [8]:
nft_sales_df.head()

,dateMinted,id,isFirstSale,lastSalePriceInETH,netRevenueInETH,netRevenuePendingInETH,netSalesInETH,netSalesPendingInETH,percentSplit,tokenIPFSPath
0,1618452786,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,False,2224.0000,1890.40000,12677.174465,2224.0000,14914.3229,NaN,QmNeqUFf3WKcXFEHgxFCrf3t8jrxyCkev6LBtRz78Ygq9u/metadata.json
1,1634633808,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,False,49.5002,42.07517,158.788500,49.5002,186.8100,NaN,QmaW2ukMCqyMNg5qqoYsusgbNL9KRnFwRKCkBt6u3AiyTN/metadata.json
2,1634808905,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100995,False,33.0000,3.30000,0.000000,33.0000,0.0000,NaN,QmdCKx9Q5b1G4vjbJ2wKSkCo3sWoUrg2hm5QTifNag5nXj/metadata.json
3,1634809019,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100997,True,NaN,0.00000,0.000000,0.0000,0.0000,NaN,QmbeU7NQaLKou2MnVDguPBJTuX1r6qAdmZ3CaSKGtE75iW/metadata.json
4,1634809215,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100998,True,NaN,0.00000,0.000000,0.0000,0.0000,NaN,QmUZ7gQDvFUWXmqTBY7FHgqVBCvzvxfNSf6vEZdVffsGpM/metadata.json


Convert epoch time to date time for date_minted

In [9]:
nft_sales_df['n_date_minted'] = [datetime.datetime.fromtimestamp(nft_sales_df['dateMinted'][x]) for x in range(len(nft_sales_df['dateMinted']))];

The "percentSplit" column contains more than 96% null and is removed. Pending ETH values are also removed.

In [10]:
1 - nft_sales_df['percentSplit'].notnull().sum() / len(nft_sales_df['percentSplit'])

0.966224715281319

In [11]:
nft_sales_df.drop(['netRevenuePendingInETH', 'netSalesPendingInETH', 'percentSplit', 'dateMinted'], axis = 1, inplace = True);

In [12]:
new_column_names = ['n_id', 'n_isFirstSale', 'n_lastSalePriceInETH', 'n_revenue', 'n_sales', 'n_tokenIPFSPath', 'n_date_minted']
old_column_names = nft_sales_df.columns
column_change = {old_column_names[col]:new_column_names[col] for col in range(len(old_column_names))}
nft_sales_df.rename(columns = column_change, inplace = True);

### Checking to make sure there are no erroneous data for sales and revenue

In [13]:
nft_sales_df['n_rs_ratio'] = nft_sales_df['n_revenue'] / nft_sales_df['n_sales'] 
nft_sales_df[nft_sales_df['n_rs_ratio'] > 1].shape

(0, 8)

In [14]:
nft_sales_df.isnull().sum()

n_id                        0
n_isFirstSale               0
n_lastSalePriceInETH    24337
n_revenue                   0
n_sales                     0
n_tokenIPFSPath             0
n_date_minted               0
n_rs_ratio              24344
dtype: int64

## 2.4 Cleaning of nft metadata

In [15]:
nft_metadata_df.head()

,tokenIPFS,name,description,image
0,QmNeqUFf3WKcXFEHgxFCrf3t8jrxyCkev6LBtRz78Ygq9u/metadata.json,"Stay Free (Edward Snowden, 2021)","This unique, signed work combines the entirety of a landmark court decision ruling the National Security Agency's mass surveillance violated the law, with the iconic portrait of the whistleblower ...",ipfs://QmTKdyzYbT9FzAs8mB4HGiLDR7o8maRP34J5iNDFZQqndQ/nft.png
1,QmaW2ukMCqyMNg5qqoYsusgbNL9KRnFwRKCkBt6u3AiyTN/metadata.json,PORTAL 07,Possession of this PORTAL entitles you to membership in the TBOA club (theblocksofart.com)\n\n10000x10000,ipfs://QmYChA4jtLhiMLPhDZv1t3xGZ1uj46yG4dtzCygUjgdhz7/nft.png
2,QmdCKx9Q5b1G4vjbJ2wKSkCo3sWoUrg2hm5QTifNag5nXj/metadata.json,PORTAL 08,Possession of this PORTAL entitles you to membership in the TBOA club (tboa.club)\n\n444(4) Edition\n\n10000x10000,ipfs://Qmbd3CzQMBPYeYBiMWe7U2bTWCyRusEjcMB3Ck84LcDbGc/nft.png
3,QmbeU7NQaLKou2MnVDguPBJTuX1r6qAdmZ3CaSKGtE75iW/metadata.json,PORTAL 09,Possession of this PORTAL entitles you to membership in the TBOA club (tboa.club)\n\n444(4) Edition\n\n10000x10000,ipfs://QmPMqidEE76so1tksfE1DMMheohB3NUzVVhZBx5VJXbRkz/nft.png
4,QmUZ7gQDvFUWXmqTBY7FHgqVBCvzvxfNSf6vEZdVffsGpM/metadata.json,PORTAL 10,Possession of this PORTAL entitles you to membership in the TBOA club (tboa.club)\n\n444(4) Edition\n\n10000x10000,ipfs://QmZATpHmc3f5vxaVAzKCveEWdNR5jDUWY9Cyqv3pMiXJbP/nft.png


In [16]:
new_column_names = ['m_tokenIPFS', 'm_name', 'm_desc', 'm_img']
old_column_names = nft_metadata_df.columns
column_change = {old_column_names[col]:new_column_names[col] for col in range(len(old_column_names))}
nft_metadata_df.rename(columns = column_change, inplace = True);

In [17]:
nft_metadata_df.isnull().sum()

m_tokenIPFS       0
m_name            3
m_desc         1238
m_img             2
dtype: int64

## 2.5 Cleaning of nft bid_data using connector "nft_nft_history_df"

In [18]:
nft_bid_df = pd.merge(nft_bid_df, nft_nft_history_df, how = "left", left_on = "id", right_on = "nft_history_id", validate = "one_to_one")

#### Generate descriptive metrics for events (pivot then flatten back to dataframe)

In [19]:
nft_bid_df_pivot = pd.pivot_table(nft_bid_df,
                                  index = ["nft_id", "event"],
                                   values = ["amountInETH"],
                                   aggfunc = {"amountInETH":[min, max, np.mean, len]}
                                  )
nft_bid_df = pd.DataFrame(nft_bid_df_pivot.to_records())

#### Only the listed, bid, sold, and private sale events are preserved, where private sale is treated as equal to sale

In [20]:
nft_bid_df['event'].value_counts()

Minted                           58862
Listed                           55755
Bid                              33969
Settled                          33837
Sold                             33837
PriceChanged                     17356
Unlisted                          8700
Burned                            7036
Transferred                       6605
PrivateSale                        701
CreatorMigrated                    346
SellerMigrated                      85
OwnerMigrated                       44
CreatorPaymentAddressMigrated       16
Name: event, dtype: int64

In [21]:
nft_bid_df = nft_bid_df[nft_bid_df['event'].isin(['Bid', 'Listed', 'Sold','PrivateSale'])]
nft_bid_df['event'].replace('PrivateSale','Sold', inplace = True);

#### Pivot once more to reduce bid, list, and sale details down to one row per nft

In [22]:
nft_bid_df = pd.pivot_table(nft_bid_df,
                            index = "nft_id",
                            columns = "event")
nft_bid_df = pd.DataFrame(nft_bid_df.to_records())

#### Tidy up columns

In [23]:
columns_to_drop =  ['("(\'amountInETH\', \'len\')", \'Listed\')',
 '("(\'amountInETH\', \'len\')", \'Sold\')',
 '("(\'amountInETH\', \'mean\')", \'Listed\')',
 '("(\'amountInETH\', \'mean\')", \'Sold\')',
 '("(\'amountInETH\', \'min\')", \'Listed\')',
 '("(\'amountInETH\', \'min\')", \'Sold\')']
nft_bid_df.drop(columns_to_drop, axis = 1, inplace = True)

In [24]:
old_column_names = nft_bid_df.columns
new_column_names =  ['n_id','n_bid_count','n_bid_max','n_list_max','n_sold_max','n_bid_max','n_bid_min']
column_change = {old_column_names[col]:new_column_names[col] for col in range(len(old_column_names))}
nft_bid_df.rename(columns = column_change, inplace = True);

In [25]:
nft_bid_df.isnull().sum()

n_id               0
n_bid_count    22208
n_bid_max      22208
n_list_max       422
n_sold_max     21670
n_bid_max      22208
n_bid_min      22208
dtype: int64

## 2.6 Combining all tables into flat table for data analysis

In [26]:
print("creator_df :" + str(creator_df.shape))
print("nft_sales_df :" + str(nft_sales_df.shape))
print("nft_metadata_df :" + str(nft_metadata_df.shape))
print("nft_bid_df :" + str(nft_bid_df.shape))

print("creator_nft_df :" + str(creator_nft_df.shape))
print("nft_nft_history_df :" + str(nft_nft_history_df.shape))

creator_df :(6000, 4)
nft_sales_df :(58830, 8)
nft_metadata_df :(58720, 4)
nft_bid_df :(56177, 7)
creator_nft_df :(58904, 2)
nft_nft_history_df :(327651, 2)


### Cleaning the connector "creator_nft_df"
Drop records where nft_id is null in "creator_nft_df", likely due to being system transactions in processing.

In [27]:
print("Before dropping null nft_id :" + str(creator_nft_df.shape))
creator_nft_df = creator_nft_df[creator_nft_df.nft_id.notnull()]
print("After dropping null nft_id :" + str(creator_nft_df.shape))

Before dropping null nft_id :(58904, 2)
After dropping null nft_id :(58871, 2)


### Combining "nft_sales_df" with "creator_df" by using connector "creator_nft_df"

In [28]:
print("Before combining :" + str(nft_sales_df.shape))
nft_combined = pd.merge(nft_sales_df, creator_nft_df, how = "left", left_on = "n_id", right_on = "nft_id", validate = "one_to_one")
nft_combined.drop('nft_id', axis = 1, inplace = True)
print("After combining :" + str(nft_combined.shape))

Before combining :(58830, 8)
After combining :(58830, 9)


In [29]:
print("Before combining :" + str(nft_combined.shape))
nft_combined = pd.merge(nft_combined, creator_df, how = "left", left_on = "creator_id", right_on = "c_id", validate = "many_to_one")
nft_combined.drop('creator_id', axis = 1, inplace = True)
print("After combining :" + str(nft_combined.shape))

Before combining :(58830, 9)
After combining :(58830, 12)


In [30]:
nft_combined = pd.merge(nft_combined, nft_metadata_df, how = "left", left_on = "n_tokenIPFSPath", right_on = "m_tokenIPFS", validate = "many_to_one")
nft_combined.drop('m_tokenIPFS', axis = 1, inplace = True)

### Further combining with "nft_bid_df", whose connector has already been attached in 2.5

In [31]:
print("Before combining :" + str(nft_combined.shape))
nft_combined = pd.merge(nft_combined, nft_bid_df, how = "left", left_on = "n_id", right_on = "n_id", validate = "one_to_one")
print("After combining :" + str(nft_combined.shape))

Before combining :(58830, 15)
After combining :(58830, 21)


### Further combining with "nft_metadata_df", and dropping any NFTs with duplicate IPFS

In [32]:
nft_combined_duplicate_IPFS = nft_combined[nft_combined['n_tokenIPFSPath'].duplicated() == True].sort_values(by = "n_tokenIPFSPath")
print("Before dropping duplicate IPFS "+ str(nft_combined.shape))
nft_combined.drop(nft_combined[nft_combined['n_tokenIPFSPath'].duplicated() == True].index, inplace = True)
print("After dropping duplicate IPFS " + str(nft_combined.shape))

Before dropping duplicate IPFS (58830, 21)
After dropping duplicate IPFS (58720, 21)


In [33]:
print("Before combining :" + str(nft_combined.shape))
nft_combined = pd.merge(nft_combined, nft_metadata_df, how = "left", left_on = "n_tokenIPFSPath", right_on = "m_tokenIPFS", validate = "one_to_one")
nft_combined.drop('m_tokenIPFS',axis = 1, inplace = True)
print("After combining :" + str(nft_combined.shape))

Before combining :(58720, 21)
After combining :(58720, 24)


## 2.7 Tidying up the final table

As the project is to develop a model for nft prices, only records where lastSalePriceInETH is not null and not zero is preserved.

In [34]:
print("Before removing nulls and zeros for sale price" + str(nft_combined.shape))
nft_combined =  nft_combined[nft_combined.n_lastSalePriceInETH.notnull()]
print("After removing nulls " + str(nft_combined.shape))
nft_combined =  nft_combined[nft_combined['n_lastSalePriceInETH']!= 0]
print("After removing zeros " + str(nft_combined.shape))

Before removing nulls and zeros for sale price(58720, 24)
After removing nulls (34446, 24)
After removing zeros (34437, 24)


Checking if data from bid table matches with nft table, then removing records with mismatch.

In [35]:
(nft_combined['n_lastSalePriceInETH']!=nft_combined['n_sold_max']).sum() 

135

In [36]:
# Come back to check how come the prices dont match for 135 records (check against all 3 sales columns)
nft_sales_mismatch = nft_combined[nft_combined['n_lastSalePriceInETH']!=nft_combined['n_sold_max']]

print("Before removing mismatch in sale " + str(nft_combined.shape))
nft_combined =  nft_combined[nft_combined['n_lastSalePriceInETH'] == nft_combined['n_sold_max']]
print("After removing mismatch in sale " + str(nft_combined.shape))

Before removing mismatch in sale (34437, 24)
After removing mismatch in sale (34302, 24)


In [37]:
print("Before dropping rows with any null values and repetitive sales columns " + str(nft_combined.shape))
nft_combined.drop(['n_sales','n_sold_max'],axis = 1, inplace = True)
nft_combined.dropna(inplace = True)
print("After dropping rows with any null values and repetitive sales columns " + str(nft_combined.shape))

Before dropping rows with any null values and repetitive sales columns (34302, 24)
After dropping rows with any null values and repetitive sales columns (32949, 22)


In [38]:
nft_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32949 entries, 0 to 58664
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   n_id                  32949 non-null  object        
 1   n_isFirstSale         32949 non-null  bool          
 2   n_lastSalePriceInETH  32949 non-null  float64       
 3   n_revenue             32949 non-null  float64       
 4   n_tokenIPFSPath       32949 non-null  object        
 5   n_date_minted         32949 non-null  datetime64[ns]
 6   n_rs_ratio            32949 non-null  float64       
 7   c_id                  32949 non-null  object        
 8   c_revenue             32949 non-null  float64       
 9   c_sales               32949 non-null  float64       
 10  c_rs_ratio            32949 non-null  float64       
 11  m_name_x              32949 non-null  object        
 12  m_desc_x              32949 non-null  object        
 13  m_img_x         

In [39]:
nft_combined.describe().T

,count,mean,std,min,25%,50%,75%,max
n_lastSalePriceInETH,32949.0,1.201182,13.919724,0.100,0.221000,0.456000,1.000000,2224.000000
n_revenue,32949.0,1.003223,11.822034,0.011,0.187085,0.382500,0.850000,1890.400000
n_rs_ratio,32949.0,0.839423,0.076182,0.100,0.850000,0.850000,0.850000,0.927778
c_revenue,32949.0,14.091949,74.878453,0.100,1.657500,3.843205,9.141325,1890.400000
c_sales,32949.0,17.334503,91.623301,0.733,1.986100,4.650000,11.220000,2224.000000
c_rs_ratio,32949.0,0.829179,0.058666,0.100,0.850000,0.850000,0.850000,0.850000
n_bid_count,32949.0,2.412000,2.915893,1.000,1.000000,1.000000,3.000000,43.000000
n_bid_max,32949.0,1.201922,13.920341,0.100,0.221000,0.456000,1.000000,2224.000000
n_list_max,32949.0,9.486055,725.354380,0.100,0.250000,0.500000,1.000000,100000.000000
n_bid_max,32949.0,0.826156,3.697942,0.100,0.200000,0.400000,0.925000,552.849126


In [40]:
nft_combined.isnull().sum()

n_id                    0
n_isFirstSale           0
n_lastSalePriceInETH    0
n_revenue               0
n_tokenIPFSPath         0
n_date_minted           0
n_rs_ratio              0
c_id                    0
c_revenue               0
c_sales                 0
c_rs_ratio              0
m_name_x                0
m_desc_x                0
m_img_x                 0
n_bid_count             0
n_bid_max               0
n_list_max              0
n_bid_max               0
n_bid_min               0
m_name_y                0
m_desc_y                0
m_img_y                 0
dtype: int64

In [41]:
nft_combined.head(20)

,n_id,n_isFirstSale,n_lastSalePriceInETH,n_revenue,n_tokenIPFSPath,n_date_minted,n_rs_ratio,c_id,c_revenue,c_sales,...,m_desc_x,m_img_x,n_bid_count,n_bid_max,n_list_max,n_bid_max,n_bid_min,m_name_y,m_desc_y,m_img_y
0,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,False,2224.0000,1890.400000,QmNeqUFf3WKcXFEHgxFCrf3t8jrxyCkev6LBtRz78Ygq9u/metadata.json,2021-04-14 22:13:06,0.850000,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.400000,2224.0000,...,"This unique, signed work combines the entirety of a landmark court decision ruling the National Security Agency's mass surveillance violated the law, with the iconic portrait of the whistleblower ...",ipfs://QmTKdyzYbT9FzAs8mB4HGiLDR7o8maRP34J5iNDFZQqndQ/nft.png,31.0,2224.0000,2.0,552.849126,2.0000,"Stay Free (Edward Snowden, 2021)","This unique, signed work combines the entirety of a landmark court decision ruling the National Security Agency's mass surveillance violated the law, with the iconic portrait of the whistleblower ...",ipfs://QmTKdyzYbT9FzAs8mB4HGiLDR7o8maRP34J5iNDFZQqndQ/nft.png
1,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,False,49.5002,42.075170,QmaW2ukMCqyMNg5qqoYsusgbNL9KRnFwRKCkBt6u3AiyTN/metadata.json,2021-10-19 04:56:48,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,...,Possession of this PORTAL entitles you to membership in the TBOA club (theblocksofart.com)\n\n10000x10000,ipfs://QmYChA4jtLhiMLPhDZv1t3xGZ1uj46yG4dtzCygUjgdhz7/nft.png,8.0,49.5002,0.1,29.538775,0.1000,PORTAL 07,Possession of this PORTAL entitles you to membership in the TBOA club (theblocksofart.com)\n\n10000x10000,ipfs://QmYChA4jtLhiMLPhDZv1t3xGZ1uj46yG4dtzCygUjgdhz7/nft.png
2,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100995,False,33.0000,3.300000,QmdCKx9Q5b1G4vjbJ2wKSkCo3sWoUrg2hm5QTifNag5nXj/metadata.json,2021-10-21 05:35:05,0.100000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,...,Possession of this PORTAL entitles you to membership in the TBOA club (tboa.club)\n\n444(4) Edition\n\n10000x10000,ipfs://Qmbd3CzQMBPYeYBiMWe7U2bTWCyRusEjcMB3Ck84LcDbGc/nft.png,1.0,33.0000,33.0,33.000000,33.0000,PORTAL 08,Possession of this PORTAL entitles you to membership in the TBOA club (tboa.club)\n\n444(4) Edition\n\n10000x10000,ipfs://Qmbd3CzQMBPYeYBiMWe7U2bTWCyRusEjcMB3Ck84LcDbGc/nft.png
6,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-104231,False,38.5000,32.725000,QmcwsRse5TswGBcNMyzUqEERpKAdv2knwkXCab28xzXqMg/metadata.json,2021-11-02 11:12:05,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,...,Possession of this PORTAL entitles you to membership in the TBOA club (tboa.club)\n\n10000x10000,ipfs://QmcReVCdz6CSsvhYNnrGQH93BEQhdatd7TNV1CivYqFer1/nft.png,5.0,38.5000,0.1,14.772000,0.1000,PORTAL 12,Possession of this PORTAL entitles you to membership in the TBOA club (tboa.club)\n\n10000x10000,ipfs://QmcReVCdz6CSsvhYNnrGQH93BEQhdatd7TNV1CivYqFer1/nft.png
8,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-63149,False,5.0000,4.250000,QmQgj6xy2cwjnH5vpQoey4EAHXddafDb5DZiAXqpVL6fco/metadata.json,2021-07-23 04:16:25,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,...,8000 x 8000,ipfs://QmbQn11K5DMk14FxLVwNyfJMWLZPy2biRGgnPwebzYGoxf/nft.png,9.0,5.0000,1.2,2.338900,1.2000,dєѕírєd ѕσlítudє,8000 x 8000,ipfs://QmbQn11K5DMk14FxLVwNyfJMWLZPy2biRGgnPwebzYGoxf/nft.png
9,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-64683,False,1.3200,1.122000,QmRUMLZTL6pGA8FhXYZHHaZdRzpkV16tGaMigb3hRmdqza/metadata.json,2021-07-26 17:33:22,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,...,8000x8000,ipfs://Qmb9oBysDQTSd8uEsBwZVqesUMVdBhQQdBvaUMPx44r5m9/nft.png,2.0,1.3200,6.0,1.260000,1.2000,bᎥᎾᏟᎬᏞᎬsᏆᎥᎪᏞ,8000x8000,ipfs://Qmb9oBysDQTSd8uEsBwZVqesUMVdBhQQdBvaUMPx44r5m9/nft.png
10,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-65069,False,23.8800,20.298000,QmXZ24m9rrTnsvKtU5jZzquonKJq1TdQxWCYz3zfzWfSKh/metadata.json,2021-07-27 12:52:05,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,...,Possession of this PO

In [42]:
datapath = "../data/processed/"
nft_combined.to_csv(datapath + 'nft_combined.csv', index = False)